# Bayesian Calibration of a SIGMA-GP Poisson process using Gibbs Sampling

M. Keller, I. Seydi, 2025

## SIGMA-GP generative model

Suppose we observe a dataset $D :$
$$
D := \{(x_i, y_i)\}_{1\leq i\leq N}
$$
which follows a time-homogeneous Poisson process over a certain period of time $T$ and a certain 2D domain $\Omega \subset \mathbb R^2,$ with a non-homogeneous spatial intensity given by: $\lambda(x,y)$ such that:
$$
N \vert \lambda \sim \mathcal P(\Lambda\times T) 
$$
$$
\left[D \vert N, \lambda\right] = \Lambda^{-N}\prod_{i=1}^N  \lambda\left(x_i, y_i \right)
$$
where: $\Lambda = \int_{x,y \in \Omega} \lambda(x, y)dxdy.$ Remember the likelihood is then given by:
$$
\left[D , N \vert \lambda\right] = \frac{\exp\left( - \Lambda \times T\right) \times T^N}{N!} \prod_{i=1}^N  \lambda\left(x_i, y_i \right)
$$

We model $\lambda$ as:
$$
\lambda(x, y) = \bar\lambda \times \sigma\circ f(x, y),
$$
where $\bar\lambda$ is known,
$$
\sigma(x) = \frac{1}{1 + \exp(-x)}
$$
and $f:\Omega \mapsto\mathbb R$ is an unknwon function we wish to estimate. 
Adopting a Bayesian perspective, we define a Gaussian process prior on $f$ with known mean function $m(x,y)$ and Gaussian covariance function $k((x, y) ; (x',y')):$
$$
f \sim GP\left(m(x,y)\ ;\ k((x, y) ; (x',y')) \right)
$$

Our goal is to sample from the posterior density of $f$ or, more precisely, of $f_T := \{f(x,y)\}_{(x,y) \in T}$ with $T$ an arbitrary finite set of test points : 
$$
\left[ f_T \vert D \right] \propto \left[ D \vert f_T \right] \times \left[ f_T\right] 
$$
$$
 \propto \mathcal P\left( N \vert \bar\lambda\int_\Omega \sigma \circ f_T\right) \times \left[D \vert N, \bar\lambda \times \sigma\circ f(x, y)\right] \times GP\left( f_T \vert m(\cdot ), k(\cdot, \cdot)\right).
$$

Except in special cases, $\int_\Omega \sigma \circ f_T$ is untractable and hence so is the target posterior. This is why, following Donner, Opper, Molkenthin etc. we resort to data augmentation as described hereafter.

###  Informative prior based on seismotectonical zonings

Furthermore, we wish to introduce a prior on the mean function $m(x, y):\mathbb R^2 \to \mathbb R^{+\star}$ based on a partition 
$$
\bigsqcup_{j=1}^J S_j = \Omega 
$$
of the search domain, which identifies zones $S_j$ of similar intensities. We use these as factors in the following linear decomposition of the GP mean function :
$$
m(x, y) := \sum_{j=1}^J \epsilon_j {\bf 1}_{\{(x, y) \in S_j\}} := U(x, y)  \epsilon,
$$
with $U(x, y) = ({\bf 1}_{\{(x, y) \in S_j\}})_{j=1:J} \in \{0, 1\}^J $
We then define a prior covariance matrix on the regressor vector ${\bf\epsilon} = (\epsilon_1, \ldots, \epsilon_J)$:
$$
{\bf\epsilon} \sim \mathcal N_J\left( 0, \Sigma_\epsilon  \right).
$$
Here we choose to model the covariance between a pair of zones $(S_j, S_{j'})$ as a function of the difference between the corresponding centers of mass $(c_j - c_{j'}),$ with 
$$
c_j = \frac{\int_{(x,y)\in S_j}(x,y)dxdy}{\int_{(x,y)\in S_j}dxdy}
$$ 
so that:
$$
\Sigma_\epsilon = \left(  k_\epsilon(c_j, c_{j'}) \right)_{1\leq j,j'\leq J}.
$$

#### Integrating $\epsilon$ out

Note that $\bf\epsilon$ can be integrated out completely from the joint density of the couple $(\bf\epsilon, f),$ by remarking that the above-defined process $m(x,y)$ is Gaussian, as a pointwise linear transform of a Gaussian vector, with mean and covariance functions given by:
$$
m(x, y) \sim GP \left( 0\ ;\ U(x, y)^\top \Sigma_\epsilon U(x', y') \right),
$$
Hence, this amounts to re-defining the GP covariance kernel by adding a non-stationary term:
$$
f \sim GP\left(0\ ;\ k((x, y) ; (x',y')) + U(x, y)^\top \Sigma_\epsilon U(x', y') \right).
$$

## Latent Poisson Process

First, we introduce $\Pi=\{(X_j, Y_j)\}_{1\leq j\leq N_\Pi}$ a latent random variable, defined as the realization of a second Poisson process which is time-homogeneous over the same period of time $T,$ and spatially non homogeneous over the same 2D domain $\Omega \subset \mathbb R^2,$ with intensity $\check\lambda(x,y)$ such that:
$$
\check\lambda(x,y) := \bar\lambda - \lambda(x,y) = \bar\lambda\left(1- \sigma\left\{ f(x, y)\right\}\right) = \bar\lambda\sigma\left\{ -f(x, y)\right\}.
$$
Notice that the superposition $D \cup \Pi,$ with size $N_{tot} := N+N_\Pi,$ follows a spatially homogeneous Poisson process since:
$$
\lambda(x,y) + \check\lambda(x,y) = \lambda(x,y) + \bar\lambda - \lambda(x,y) = \bar\lambda,
$$
Hence 
$$
N_{tot}:=N+N_\Pi \sim \mathcal P\left( \bar\lambda \times T \times \vert\Omega\vert \right).
$$
Hence $N_{tot}$ is easy to simulate, following the above Poisson law. $\Pi$ is also easy to simulate given $f,$ by thinning: simply draw $N_{tot}$ uniform 2D points $(X_j, Y_j)$ within $\Omega,$ compute $f_j := f(X_j, Y_j)$ then allocate each point $j$ to $\Pi$ with probability:
$$
\mathbb P\left[  (X_j, Y_j) \in \Pi \vert f_j \right] = \frac{\sigma\left\{ -f_j\right\}}{\sigma\left\{ -f_j\right\} + \sigma\left\{ f_j\right\}} = \sigma\left\{ -f_j\right\}.
$$
Once this is done, re-define and re-order the $(X_j, Y_j)$ such that $\{(X_j, Y_j)\}_{1\leq j\leq N} = D$ and $\{(X_j, Y_j)\}_{N\leq j+1\leq N_{tot}} = \Pi$


## Latent Polya-Gamma Process (PGP)

We now introduce a second set of latent variables ${\boldsymbol\omega} = (\omega_j)_{1\leq j\leq N_{tot}},$ such that, *a priori*:
$$
\omega_j \vert N_{tot} \stackrel{iid}{\sim} PG(1, 0)
$$
and re-define arbitrarily the augmented conditional density of $D \cup \Pi $ given $ f, \boldsymbol\omega$:
$$
\left[ D \cup \Pi \vert (f_j)_{j=1, \ldots, N_{tot}}, {\bf\omega} \right] = 
\frac{\exp\left(-\bar\lambda\times T\times \vert\Omega\vert\right)}{N_{tot}!}
%\left(\bar\lambda\times T\times \vert\Omega\vert\right)^{N_{tot}}
\times\prod_{j=1}^N \frac{\bar\lambda}{2}\exp\left(\frac{f_j}{2}-\frac{f_j^2}{2}\omega_j\right) 
\times \prod_{j=N+1}^{N_{tot}}\frac{\bar\lambda}{2} \exp\left(-\frac{f_j}{2}-\frac{f_j^2}{2}\omega_j\right),
$$
then (thanks to elaborate Laplace-transformation wizardry) we can show that, by integrating $\boldsymbol\omega$ out, we indeed recover:
$$
\left[ D \cup \Pi \vert (f_j)_{j=1, \ldots, N_{tot}}, N \right] = 
\frac{\exp\left(-\bar\lambda\times T\times \vert\Omega\vert\right)}{N! (N_{tot}-N)!}
\prod_{j=1}^N \bar\lambda \sigma(f_j) \times \prod_{j=N+1}^{N_{tot}}\bar\lambda \sigma(-f_j).
$$
$$
= [D,N \vert \lambda] \times [\Pi,N_{tot}-N \vert \lambda].
$$

### Conditional conjugacy

Plus, the PGP model is conjugate to the SIGMA-GP model, in that the $w_j$'s are *a posteriori* independent conditional on $D\cup \Pi$ and the $f_j$, for $j = 1,\ldots, N_{tot} :$
$$
w_j \vert D \cup \Pi, f_j \stackrel{ind}{\sim} PG(1,\vert f_j\vert).
$$

This property is key to including PGP variables as a block in a Gibbs sampling scheme.



## Summary

The full data-augmented model contains the following variables, oserved or latent:

- $D$: the observed points
- $\Pi$ the latent points
- $\boldsymbol \omega = \boldsymbol \omega_D \cup \boldsymbol \omega_\Pi$ the latent PGP at observed and latent points
- $\boldsymbol f = \boldsymbol f_D \cup \boldsymbol f_\Pi$ the latent GP at observed and latent points

Then the complete augmented likelihood is given by: 
$$
\left[ D, \boldsymbol f, \boldsymbol \omega_D, \Pi,, \boldsymbol \omega_\Pi, N_{tot} \vert \bar\lambda \right]
=
\left[ D \vert \boldsymbol f_D, {\boldsymbol\omega_D} \right] \left[ \Pi \vert \boldsymbol f_\Pi, {\boldsymbol\omega_D} \right] \left[ \boldsymbol f \right] \left[{\boldsymbol\omega} \right] 
$$
$$= 
\frac{\exp\left(-\bar\lambda\times T\times \vert\Omega\vert\right)}{N_{tot}!}
%\left(\bar\lambda\times T\times \vert\Omega\vert\right)^{N_{tot}}
\times\prod_{j=1}^N \frac{\bar\lambda}{2}\exp\left(\frac{f_j}{2}-\frac{f_j^2}{2}\omega_j\right) 
\times \prod_{j=N+1}^{N_{tot}}\frac{\bar\lambda}{2} \exp\left(-\frac{f_j}{2}-\frac{f_j^2}{2}\omega_j\right),
$$
$$
\times \boldsymbol N\left( \boldsymbol f \vert \boldsymbol m, \boldsymbol K \right) \times \prod_{j=1}^{N} PG(\omega_j \vert 1, 0) \times \prod_{j=N+1}^{N_{tot}} PG(\omega_j \vert 1, 0)
$$

## Blocked Gibbs Algorithm

The idea is to divide the above variables into a certain number of blocks, such that the complete conditional density of each block can be simulated exactly. Here are the blocks that we suggest: 

- $\bf f := \boldsymbol f_D \cup \boldsymbol f_\Pi $ the latent GP evaluated at observed and latent points;
- $\Pi$ : the latent Poisson process and the associated latent Polya-Gamma process;
- $\boldsymbol\omega = \boldsymbol \omega_D \cup \boldsymbol \omega_\Pi $ the latent PGP at the observed and latent points

The Gibbs algorithm generates a Markov chain $(\boldsymbol\theta^{(t)})_{t=1,\ldots,T}$ with $\boldsymbol\theta^{(t)}:=\left(\boldsymbol f^{(t)}, \Pi^{(t)},\boldsymbol \omega^{(t)} \right),$ given a starting point $\boldsymbol\theta^{(1)},$ ideally simulated from the prior law on $\theta,$ and iterating the following conditional updating steps.

### Remark

By analyzing the details of the updating steps below, it becomes clear that only the initial values $\Pi^{(1)}, \boldsymbol\omega^{(1)}$ really need to be generated, the remaining components, corresponding to $\boldsymbol f^{(1)},$ can be set arbitrarily. We suggest to simulate $N_{tot}^{(1)} \sim \mathcal P(\bar\lambda\ T\ \vert\Omega\vert)\times \boldsymbol 1_{\left\{N_{tot}^{(1)}\geq N\right\}},$ simulate $(X_i^{(1)}, Y_i^{(1)})\stackrel{iid}{\sim}\mathcal U(\Omega)$ for $i=N+1, \ldots, N_{tot}$ then set $\Pi^{(1)}:=\{(X_i^{(1)}, Y_i^{(1)})\}_{i=N+1,\ldots, N_{tot}^{(1)}}$ and finally $\omega_i^{(1)} \stackrel{iid}{\sim} PG(1,0)$ for $i=1,\ldots, N_{tot}^{(1)}$


We now assume we have generated $\boldsymbol \theta^{(t)}$ for some $t\geq 1,$ and detail step by step how to obtain the updated vector $\boldsymbol \theta^{(t+1)}.$

### Updating the latent GP evaluated at all points

The latent GP prior is conjugate to the PGP model, so that the conditional posterior used to simulate the updated GP $\boldsymbol f^{(t+1)}$ is the multivariate normal:
$$
\boldsymbol f^{(t+1)} \vert D, \Pi^{(t)}, \boldsymbol \omega^{(t)}, \boldsymbol \epsilon^{(t)} \sim \mathcal N\left(\left(\boldsymbol \Omega^{(t)} + {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1}\right)^{-1}\left( {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1}\boldsymbol m^{(t)} + \boldsymbol u\ \right);\ \left(\boldsymbol \Omega^{(t)} + {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1}\right)^{-1}\right),
$$
with 
$$
\boldsymbol \Omega^{(t)} = diag\left(\boldsymbol\omega^{(t)}\right),
$$ 
$$
\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}= \left( k((X_i^{(t)}, Y_i^{(t)}), (X_j^{(t)}, Y_j^{(t)})^{(t)}) \right)_{\substack{i=1:N_{tot}^{(t)}\\ j=1:N_{tot}^{(t)}}}
$$
$$
\boldsymbol m^{(t)} = \boldsymbol U^{(t)} \boldsymbol \epsilon^{(t)}
$$
$$
\boldsymbol U^{(t)} = \left( \boldsymbol 1_{S_j}\left(X_i^{(t)}, Y_i^{(t)}\right) \right)_{\substack{ i=1:N_{tot}^{(t)} \\ j=1:J }}
$$
$$
\boldsymbol X^{(t)}, \boldsymbol Y^{(t)} = D \cup \Pi^{(t)}
$$
and $\boldsymbol u^{(t)} := (u_1^{(t)}, \ldots, u^{(t)}_{N_{tot}^{(t)}}),$ $u_i^{(t)} = \frac{1}{2}$ if $i \leq N$ and $u_i^{(t)} = -\frac{1}{2}$ if $i > N.$ 



<!-- #### Adding zoning prior information

As explained earlier, taking into account the zoning infromation into the latent GP prior can be done simply by modifying the prior mean and variance function, leading to the following modified conditional posterior:
$$
\boldsymbol f^{(t+1)} \vert D, \Pi^{(t)}, \boldsymbol \omega^{(t)} \sim \mathcal N\left(\left(\boldsymbol \Omega^{(t)} + {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1}\right)^{-1}\boldsymbol u\ ;\ \left(\boldsymbol \Omega^{(t)} + {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1}\right)^{-1}\right),
$$
$$
\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}= \left( k((X_i^{(t)}, Y_i^{(t)}), (X_j^{(t)}, Y_j^{(t)})^{(t)}) + U(X_i^{(t)}, Y_i^{(t)})^\top\ \Sigma_\epsilon\ U(X_j^{(t)}, Y_j^{(t)}) \right)_{\substack{i=1:N_{tot}^{(t)}\\ j=1:N_{tot}^{(t)}}}
$$
$$
\boldsymbol X^{(t)}, \boldsymbol Y^{(t)} = D \cup \Pi^{(t)}.
$$

Note that the prior mean has been set to zero, while a non-stationary term has been added to the GP covariance kernel. -->



### Updating the latent PP... and corresponding GP marginals!

Here we introduce the following *auxiliary* latent variables:
- $\Pi^\ast = ((X_i^\ast, Y_i^\ast))_{1\leq i\leq N^\ast})$  the realization of a spatially and temporally homogeneous Poisson process over spatial domain $\Omega$ and period of time $T$, with intensity $\bar\lambda.$ This means that:
$$
N^\ast \sim \mathcal P(\bar\lambda T \vert \Omega\vert)
$$
$$
X_i^\ast, Y_i^\ast \vert N^\ast \stackrel{iid}{\sim} \mathcal U(\Omega),\quad i=1,\ldots, N^\ast
$$
- $\boldsymbol f^\ast = (f(X_i^\ast, Y_i^\ast))_{1\leq i\leq N^\ast}$ the GP evaluated at the auxiliary set of points. The full conditional density of this block (given all the other blocks) is multivariate normal, following the well-known "sandwich" formula:
$$
\boldsymbol f^\ast  \vert \boldsymbol f^{(t+1)}, \Pi^{(t)}, \Pi^\ast, D \sim \mathcal N\left(
\boldsymbol m^\ast + \boldsymbol K^{(t)}_{\boldsymbol f^\ast,\boldsymbol f} {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1} (\boldsymbol f^{(t+1)} - \boldsymbol m{(t)})\ ;\ \boldsymbol K_{\boldsymbol f^\ast,\boldsymbol f^\ast} - \boldsymbol K_{\boldsymbol f^\ast,\boldsymbol f}^{(t)} {\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{(t)}}^{-1}  \boldsymbol K_{\boldsymbol f,\boldsymbol f^\ast}^{(t)} \right),
$$
with
$$
\boldsymbol K_{\boldsymbol f^\ast, \boldsymbol f}^{(t)} = \left({\boldsymbol K_{\boldsymbol f, \boldsymbol f^\ast}^{(t)}}\right)^\top = \left( k((X_i^\ast, Y_i^\ast), (X_j^{(t)}, Y_j^{(t)})^{(t)}) \right)_{\substack{i=1:N^\ast\\ j=1:N_{tot}^{(t)}}}.
$$

$\Pi^{(t+1)}$ is then recovered based on $\Pi^\ast$ and $\boldsymbol f^\ast$ through a *thinning* argument, meaning that each point $(X_i^\ast, Y_i^\ast)$ for $i=1,\ldots, N^\ast$ is allocated to $\Pi^{(t+1)}$ with probability: $\sigma (-f_i^\ast),$ where $f_i^\ast:=f(X_i^\ast, Y_i^\ast).$

To summarize, the conditional density $\Pi^{(t+1)} \vert D, \boldsymbol f^{(t+1)}$ is obtained as a marginal of the joint posterior conditional 
$$
\left[\Pi^{(t+1)}, \boldsymbol f^\ast, \Pi^\ast \vert D, \boldsymbol f^{(t+1)}, \boldsymbol \Pi^{(t)} \right] = 
\left[\Pi^{(t+1)} \vert \boldsymbol f^\ast, \Pi^\ast  \right] \times
\left[\boldsymbol f^\ast  \vert \Pi^\ast,  \boldsymbol f^{(t+1)}, \Pi^{(t)} \right] \times
\left[\Pi^\ast \right]
$$ 

which can be simulated by thinning, using the following steps:
1. Sample the total number of points of the latent homogeneous Poisson process: $N^\ast \vert D \sim \mathcal P \left(  \bar\lambda \times T \times \vert \Omega \vert \right).$
2. For $i=1, \ldots, N^\ast$: simulate $(X_i^\ast, Y_i^\ast)$ uniformly over $\Omega,$ yielding $\Pi^\ast$
3. Simulate $\boldsymbol f^\ast$ using the above conditional density $\boldsymbol f^\ast  \vert \boldsymbol f^{(t+1)},  \Pi^\ast, \Pi^{(t)}$
4. For $i=1, \ldots, N^\ast$: allocate point $(X_i^\ast, Y_i^\ast)$ to $\Pi^{(t+1)}$ with probability: $1-\sigma (f_i^\ast) = \sigma (-f_i^\ast)$;
6. Modify accordingly: $N_{tot}^{(t+1)} := N + \vert\Pi^{(t+1)}\vert$ and $\boldsymbol f_\Pi^{(t+1)}$ in $\boldsymbol f^{(t+1)} = \boldsymbol f_D^{(t+1)}\cup\boldsymbol f_\Pi^{(t+1)},$ from its current value $\boldsymbol f^{(t+1)}_{\Pi^{(t)}}=(f^{(t+1)}(X_i^{(t)}, Y_i^{(t)}))_{i=N+1,\ldots,N_{tot}^{(t)}}$ to its new value $\boldsymbol f^{(t+1)}_{\Pi^{(t+1)}} = (f_i^\ast)_{i=N+1,\ldots,N_{tot}^{(t+1)}}.$

<!-- #### Incorporating prior information
  -->

### Updating the PGP at all points (observed & latent)

The posterior conditional density of $w_i^{(t+1)}$ for $i=1, \ldots, N_{tot}$ is given by:
$$
\omega_i^{(t+1)} \vert f_i^{(t+1)} \stackrel{ind}{\sim} PG(1, \vert f_i^{(t+1)}\vert).
$$

### Updating the zone's effects'

$\boldsymbol \epsilon$'s posterior conditional is Gaussian, with conditional mean and covariance:
$$
\boldsymbol \epsilon^{(t+1)} \vert \boldsymbol f^{(t+1)}, D, \boldsymbol \Pi^{(t+1)} \sim \mathcal N\left( 
\left( \Sigma_\epsilon^{-1} + {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1} \boldsymbol U^{(t+1)}\right)^{-1} {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1}\boldsymbol f^{(t+1)}\ ;\
\left( \Sigma_\epsilon^{-1} + {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1} \boldsymbol U^{(t+1)}\right)^{-1}
\right)
$$


## Implementation details

### Dimension of the MCMC chain

We note $\boldsymbol\theta^{(t)}:=\left(\boldsymbol f^{(t)}, \boldsymbol \omega^{(t)}, \Pi^{(t)}, N_{tot}^{(t)}, \boldsymbol \varepsilon^{(t)} \right)_{1\leq t\leq T}$ the MCMC chain generated by the Gibbs sampler. Note that, at each step $t$, the chain has dimension: $d^{(t)} = 4 N_{tot}^{(t)} - 2N  + 1 + J.$

However, using Open TURNS, the dimension of the MCMC chain must be fixed in advance. Therefore, we need to choose a reasonable upper bound for $N_{tot}^{(t)},$ let's call it $N_ {max}.$ Remember that in principle $N_{tot}$'s law is close to the Poisson $\mathcal P\left( \bar\lambda \times T \times \vert \Omega \vert \right),$ hence we suggest using the $99.9$-th percentile of this law.

Open TURNS also requires the Gibbs blocks to be allocated once and for all, meaning that the blocks do not change from one iteration to the other. To do this, we must pre-allocate components of the parameter vector $\boldsymbol\theta$ to each block. We chose the following allocation policy:
$$
\boldsymbol\theta_{1:N_{ {max}}} := \boldsymbol f 
$$
$$
\boldsymbol\theta_{N_{ {max}}+1:2N_{ {max}}} := \boldsymbol \omega 
$$
$$
\boldsymbol\theta_{2N_ {max}+1:4N_ {max}-2N} := \Pi 
$$
$$
\theta_{4N_ {max} - 2N+1} :=  N_{tot}
$$
$$
\boldsymbol\theta_{4N_{max}-2N + 1 + 1: 4N_{max}-2N + 1 + J} := \boldsymbol \epsilon 
$$

This relies on the assumption that, for all $t=1,\ldots, T$ we have: $N_{tot}^{(t)} \leq N_ {max},$ so that the last $N_ {max} - N_{tot}^{(t)}$ components of the latent variables $\Pi$, $\boldsymbol f_\Pi$ and $\boldsymbol \omega_\Pi$ may take arbitrary values. On the other hand, if per chance $N_{tot}^{(t)} > N_ {max},$ the latent processes are truncated, which may induce some residual bias on the final estimation, showing that some care must be taken when calibrating $N_ {max}.$


### Efficient conditional Normal simulation

$$
\boldsymbol f \vert D, \Pi, \boldsymbol \omega \sim \mathcal N\left(\left(\boldsymbol \Omega + \boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1}\right)^{-1}\left( \boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1} \boldsymbol m + \boldsymbol u\ \right)\ ;\ \left(\boldsymbol \Omega + \boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1}\right)^{-1}\right),
$$
with $\boldsymbol \Omega = diag\left(\boldsymbol\omega\right),$ and $\boldsymbol u := (u_1, \ldots, u_{N_{tot}}),$ $u_i = \frac{1}{2}$ if $i \leq N$ and $u_i = -\frac{1}{2}$ if $i > N.$

Let $LL^\top = \boldsymbol K_{\boldsymbol f, \boldsymbol f}$ be $\boldsymbol K_{\boldsymbol f, \boldsymbol f}$'s Cholesky decomposition, so that: ${L^{-1}}^\top L^{-1} = \boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1}$

Then, owing to Woodbury's formula:
$$
\left(\boldsymbol \Omega + \boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1}\right)^{-1} = 
\left(\boldsymbol \Omega + {L^{-1}}^\top L^{-1}\right)^{-1} = 
\boldsymbol\Omega^{-1} - \boldsymbol\Omega^{-1} 
\left( 
\boldsymbol I_{N_{tot}} + L^{-1}\boldsymbol\Omega^{-1}{L^{-1}}^\top
\right)^{-1}
\boldsymbol\Omega^{-1}
$$

Remember that, to simulate $X \sim \mathcal N(\mu, \Sigma),$ with $\mu \in \mathbb R^d,$ $\Sigma \in \mathbb R^{d}\times\mathbb R^{d}$ def. pos., a numerically sensible strategy is to notice that:
$$
Z := \Sigma^{-1/2}(X - \mu) \sim \mathcal N(0, I_d),
$$
and that $\Sigma^{-1/2} = L,$ with $L$ the Cholesky decomposition of $\Sigma^{-1}$ such that: $\Sigma^{-1}=LL^\top.$ Hence $X$ can be simulated as:
$$
X := L^{-1}Z + \mu
$$
In the current case this implies $LL^\top = \Sigma^{-1} = \boldsymbol \Omega + \boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1}.$ Besides we also have:
$$
\mu = \Sigma\left(\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1} \boldsymbol m + \boldsymbol u\right)
= (L^{-1})^\top L^{-1}\left(\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1} \boldsymbol m + \boldsymbol u\right)
$$
So that finally:
$$
X := L^{-1}Z + (L^{-1})^\top L^{-1}\left(\boldsymbol K_{\boldsymbol f, \boldsymbol f}^{-1} \boldsymbol m + \boldsymbol u\right)
$$

Likewise, to simulate from the posterior conditional density of $\boldsymbol \epsilon,$ given by:
$$
\boldsymbol \epsilon^{(t+1)} \vert \boldsymbol f^{(t+1)}, D, \boldsymbol \Pi^{(t+1)} \sim \mathcal N\left( 
\left( \Sigma_\epsilon^{-1} + {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1} \boldsymbol U^{(t+1)}\right)^{-1} {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1} \boldsymbol f^{(t+1)}\ ;\
\left( \Sigma_\epsilon^{-1} + {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1} \boldsymbol U^{(t+1)}\right)^{-1}
\right),
$$
one needs to compute the Cholesky decompositions $J J^\top = K^{(t+1)}_{\boldsymbol f, \boldsymbol f}$ and $L L^\top = \left( \Sigma_\epsilon^{-1} + {\boldsymbol U^{(t+1)}}^\top {K^{(t+1)}_{\boldsymbol f, \boldsymbol f}}^{-1} \boldsymbol U^{(t+1)}\right)$ of $\boldsymbol \epsilon$'s precision matrix, then simulate Gaussian white noise $Z \sim \mathcal N\left( \boldsymbol 0_J\ ;\ \boldsymbol I_J\right),$ and then compute
$$
\boldsymbol \epsilon^{(t+1)} \vert \boldsymbol f^{(t+1)}, D, \boldsymbol \Pi^{(t+1)} \stackrel{\mathcal L}{=} L^{-1}Z + (L^{-1})^\top L^{-1} {\boldsymbol U^{(t+1)}}^\top {J^{-1}}^\top J^{-1} \boldsymbol f^{(t+1)}
$$

In [1]:
##########################
#    Necessary imports   #
##########################

import os
import openturns as ot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import time
import scipy.optimize as so
import scipy.stats as st
import statsmodels.tsa.stattools as stattools
from polyagamma import random_polyagamma

ot.RandomGenerator.SetSeed(0) # Make results reproducible by freezing Open TURNS's random generator's seed
np.random.seed(0) # Make results reproducible by freezing Numpy's random generator's seed


/home/H01971/miniforge3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#################
# Simulate data #
#################

# Assuming square domain [0,1]*[0,1] (surface 1)
# and null trend

lambdaBar = 100
T = 10

sigmoid = ot.SymbolicFunction(['z'], ['1/(1+exp(-z))'])
def trend(X):
    return [0]

cov = ot.SquaredExponential([0.5, 0.5], [1.0])
m = ot.PythonFunction(2, 1, trend)

# Use thinning


In [3]:
N_star = int(ot.Poisson(lambdaBar * T).getRealization()[0])
print(N_star)

1034


In [4]:
XY_star = np.array(ot.ComposedDistribution([ot.Uniform()]*2).getSample(N_star))
mesh = ot.Mesh(XY_star)
mTrend = ot.TrendTransform(m, mesh)
F = ot.GaussianProcess(mTrend, cov, mesh)
field_function = ot.PythonFieldFunction(mesh, 1, mesh, 1, sigmoid)
process = ot.CompositeProcess(field_function, F)     
field_f = process.getRealization()
field_f

class=Field name=Unnamed description=[v0,v1,y0] implementation=class=FieldImplementation name=Unnamed mesh=class=Mesh name=Unnamed dimension=2 vertices=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=1034 dimension=2 data=[[-0.729447,0.694488],[-0.934994,0.752554],[-0.305886,-0.199093],...,[-0.319469,-0.833137],[-0.333557,-0.395031],[-0.473865,0.403297]] simplices=[] values=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=1034 dimension=1 description=[y0] data=[[0.154713],[0.227191],[0.416696],...,[0.20724],[0.440015],[0.14481]]